# Chapter 3. 한국어 전처리

해당 챕터 내용을 공부한 흔적들을 남겨놓은 노트북이며, 코드를 정리한 결과는 py 파일로 남겨놓았습니다.

## 3.1 데이터 확보

### 3.1.1 한국어 위키백과

In [93]:
corpus_fname = '../kowiki-latest-pages-articles.xml'
with open(corpus_fname,'r',encoding='utf-8') as f1:
    for i,line in enumerate(f1):
        print(line)
        if i==100:break

<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.10/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.10/ http://www.mediawiki.org/xml/export-0.10.xsd" version="0.10" xml:lang="ko">

  <siteinfo>

    <sitename>위키백과</sitename>

    <dbname>kowiki</dbname>

    <base>https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EB%8C%80%EB%AC%B8</base>

    <generator>MediaWiki 1.34.0-wmf.24</generator>

    <case>first-letter</case>

    <namespaces>

      <namespace key="-2" case="first-letter">미디어</namespace>

      <namespace key="-1" case="first-letter">특수</namespace>

      <namespace key="0" case="first-letter" />

      <namespace key="1" case="first-letter">토론</namespace>

      <namespace key="2" case="first-letter">사용자</namespace>

      <namespace key="3" case="first-letter">사용자토론</namespace>

      <namespace key="4" case="first-letter">위키백과</namespace>

      <namespace key="5" case="first-letter

In [97]:
from gensim.corpora import WikiCorpus
from gensim.utils import to_unicode
import re

In [98]:

WIKI_REMOVE_CHARS = re.compile("'+|(=+.{2,30}=+)|__TOC__|(ファイル:).+|:(en|de|it|fr|es|kr|zh|no|fi):|\n", re.UNICODE)
WIKI_SPACE_CHARS = re.compile("(\\s|゙|゚|　)+", re.UNICODE)
EMAIL_PATTERN = re.compile("(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)", re.UNICODE)
URL_PATTERN = re.compile("((ftp|http|https):\/\/)?(www.)?(?!.*(ftp|http|https|www.))[a-zA-Z0-9_-]+(\.[a-zA-Z]+)+((\/)[\w#]+)*(\/\w+\?[a-zA-Z0-9_]+=\w+(&[a-zA-Z0-9_]+=\w+)*)?$", re.UNICODE)
WIKI_REMOVE_TOKEN_CHARS = re.compile("(\\*$|:$|^파일:.+|^;)", re.UNICODE)
MULTIPLE_SPACES = re.compile(' +', re.UNICODE)


def tokenize(content, token_min_len=2, token_max_len=100, lower=True):
    content = re.sub(EMAIL_PATTERN, ' ', content)  # remove email pattern
    content = re.sub(URL_PATTERN, ' ', content) # remove url pattern
    content = re.sub(WIKI_REMOVE_CHARS, ' ', content)  # remove unnecessary chars
    content = re.sub(WIKI_SPACE_CHARS, ' ', content)
    content = re.sub(MULTIPLE_SPACES, ' ', content)
    tokens = content.replace(", )", "").split(" ")
    result = []
    for token in tokens:
        if not token.startswith('_'):
            token_candidate = to_unicode(re.sub(WIKI_REMOVE_TOKEN_CHARS, '', token))
        else:
            token_candidate = ""
        if len(token_candidate) > 0:
            result.append(token_candidate)
    return result

In [99]:
wiki = WikiCorpus(corpus_fname,tokenize)

TypeError: '<' not supported between instances of 'function' and 'int'

### 3.1.2 KorQuAD

In [81]:
import json
corpus_fname = 'data/raw/KorQuAD_v1.0_train.json'
output_fname = 'data/processed/processed_korquad_train.txt'

In [26]:
with open(corpus_fname,'r') as f1:
    dataset_json = json.load(f1)
    dataset = dataset_json['data']
    print(dataset[0]['paragraphs'][0])

{'qas': [{'answers': [{'text': '교향곡', 'answer_start': 54}], 'id': '6566495-0-0', 'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'}, {'answers': [{'text': '1악장', 'answer_start': 421}], 'id': '6566495-0-1', 'question': '바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?'}, {'answers': [{'text': '베토벤의 교향곡 9번', 'answer_start': 194}], 'id': '6566495-0-2', 'question': '바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?'}, {'answers': [{'text': '파우스트', 'answer_start': 15}], 'id': '6566518-0-0', 'question': '1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?'}, {'answers': [{'text': '합창교향곡', 'answer_start': 354}], 'id': '6566518-0-1', 'question': '파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?'}, {'answers': [{'text': '1839', 'answer_start': 0}], 'id': '5917067-0-0', 'question': '바그너가 파우스트를 처음으로 읽은 년도는?'}, {'answers': [{'text': '파리', 'answer_start': 410}], 'id': '5917067-0-1', 'question': '바그너가 처음 교향곡 작곡을 한 장소는?'}, {'answers': [{'text': '드레스덴', 'answer_start': 534}], 'id': '5917067-0-2', 'question': '바그너의 1악장의 초연은 어디서 연주되었는가?'}], 'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고

In [34]:
f1 = open(corpus_fname,'r') 
dataset_json = json.load(f1)
dataset = dataset_json['data']
f1.close()

In [40]:
dataset[0].keys()

dict_keys(['paragraphs', 'title'])

In [42]:
dataset[0]['paragraphs'][0]

{'qas': [{'answers': [{'text': '교향곡', 'answer_start': 54}],
   'id': '6566495-0-0',
   'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'},
  {'answers': [{'text': '1악장', 'answer_start': 421}],
   'id': '6566495-0-1',
   'question': '바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?'},
  {'answers': [{'text': '베토벤의 교향곡 9번', 'answer_start': 194}],
   'id': '6566495-0-2',
   'question': '바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?'},
  {'answers': [{'text': '파우스트', 'answer_start': 15}],
   'id': '6566518-0-0',
   'question': '1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?'},
  {'answers': [{'text': '합창교향곡', 'answer_start': 354}],
   'id': '6566518-0-1',
   'question': '파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?'},
  {'answers': [{'text': '1839', 'answer_start': 0}],
   'id': '5917067-0-0',
   'question': '바그너가 파우스트를 처음으로 읽은 년도는?'},
  {'answers': [{'text': '파리', 'answer_start': 410}],
   'id': '5917067-0-1',
   'question': '바그너가 처음 교향곡 작곡을 한 장소는?'},
  {'answers': [{'text': '드레스덴', 'answer_start': 534}],
   'id': '5917067-0-2',
   'question': '바그너의 1

In [43]:
dataset[0]['paragraphs'][0]['context']

'1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.'

In [44]:
dataset[0]['paragraphs'][0]['qas']

[{'answers': [{'text': '교향곡', 'answer_start': 54}],
  'id': '6566495-0-0',
  'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'},
 {'answers': [{'text': '1악장', 'answer_start': 421}],
  'id': '6566495-0-1',
  'question': '바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?'},
 {'answers': [{'text': '베토벤의 교향곡 9번', 'answer_start': 194}],
  'id': '6566495-0-2',
  'question': '바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?'},
 {'answers': [{'text': '파우스트', 'answer_start': 15}],
  'id': '6566518-0-0',
  'question': '1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?'},
 {'answers': [{'text': '합창교향곡', 'answer_start': 354}],
  'id': '6566518-0-1',
  'question': '파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?'},
 {'answers': [{'text': '1839', 'answer_start': 0}],
  'id': '5917067-0-0',
  'question': '바그너가 파우스트를 처음으로 읽은 년도는?'},
 {'answers': [{'text': '파리', 'answer_start': 410}],
  'id': '5917067-0-1',
  'question': '바그너가 처음 교향곡 작곡을 한 장소는?'},
 {'answers': [{'text': '드레스덴', 'answer_start': 534}],
  'id': '5917067-0-2',
  'question': '바그너의 1악장의 초연은 어디서 연주되었는가?'}]

In [53]:
for data in dataset[0]['paragraphs'][0]['qas']:
    print(data['question'],data['answers'][0]['text'])

바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가? 교향곡
바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가? 1악장
바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가? 베토벤의 교향곡 9번
1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은? 파우스트
파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은? 합창교향곡
바그너가 파우스트를 처음으로 읽은 년도는? 1839
바그너가 처음 교향곡 작곡을 한 장소는? 파리
바그너의 1악장의 초연은 어디서 연주되었는가? 드레스덴


위의 시도를 토대로 다음과 같은 최종 버전 작성

In [83]:
with open(corpus_fname,'r',encoding='utf-8') as f1, open(output_fname,'w',encoding='utf-8') as f2:
    dataset_json = json.load(f1)
    dataset = dataset_json['data']
    for data in dataset:
        w_lines = []
        for paragraph in data['paragraphs']:
            w_lines.append(paragraph['context'])
            for qa in paragraph['qas']:
                q_text = qa['question']
                for a in qa['answers']:
                    a_text = a['text']
                    w_lines.append(q_text+" "+a_text)
        for line in w_lines:
            f2.writelines(line+"\n")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### 3.1.3 네이버 영화 리뷰 말뭉치

In [79]:
corpus_path = 'data/raw/ratings_train.txt'
output_fname = 'data/processed/processed_ratings.txt'
with_label = False

In [80]:
with open(corpus_path,'r',encoding='utf-8') as f1:
    i = 0 
    for line in f1:
        print(line)
        i += 1
        if i==5:break

id	document	label

9976970	아 더빙.. 진짜 짜증나네요 목소리	0

3819312	흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나	1

10265843	너무재밓었다그래서보는것을추천한다	0

9045019	교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정	0



In [76]:
with open(corpus_path,'r',encoding='utf-8') as f1,\
        open(corpus_fname,'w',encoding='utf-8') as f2:
    i = 0 
    next(f1)
    for line in f1:
        _, sentence, label = line.strip().split('\t')

        if not sentence: continue
        if with_label:
            f2.writelines(sentence+'\u241E' + label + '\n')
        else:
            f2.writelines(sentence+'\n')
        i += 1
        if i==5:break

In [77]:
with open(corpus_fname,'r',encoding='utf-8') as f2:
    for line in f2:
        print(line)

어릴때보고 지금다시봐도 재밌어요ㅋㅋ

디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.

폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.

와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런게 진짜 영화지

안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.

